In [ ]:
'''
  Function to find Minimum Spanning Tree (MST) of an un-/weighted undirected graph 
    based on Prim Algorithm using min-heap
  Time complexity = O(E.log(V))

  Parameters:
  -----------
    graph: defaultdict
           An undirected graph dictionary: { u: {v1: w1, v2: w2} }
    V    : integer
           Number of vertex

  Returns:
  --------
    mst    : list
             An undirected Minimum Spanning Tree list of edges [u, v]
    minCost: integer
             Total weights of MST

  Examples:
  --------- 
      Undirected graph            Minimum Spanning Tree
            10                            10
          0-----1                       0-----1
          |\    |                        \  
          | \   |     Prim Algorithm      \       Total weights of MST 
         6|  \5 |15  ================>     \5     = 10 + 5 + 4 = 19
          |   \ |                           \
          |    \|                            \
          2-----3                       2-----3
             4                             4 

    >>> graph = {0: {1: 10, 2: 6, 3: 5}, 1: {0: 10, 3: 15}, 2: {0: 6, 3: 4}, 3: {0: 5, 1: 15, 2: 4}}
    >>> V = 4
    >>> print(MST_Prim(graph, V))
    ([[0, 1], [3, 2], [0, 3]], 19)

  References:
    https://leetcode.com/problems/network-delay-time/discuss/329376/efficient-oe-log-v-python-dijkstra-min-heap-with-explanation
    https://www.geeksforgeeks.org/prims-mst-for-adjacency-list-representation-greedy-algo-6/?ref=rp
    https://www.youtube.com/watch?v=oP2-8ysT3QQ&list=PLrmLmBdmIlpu2f2g8ltqaaCZiq6GJvl1j&index=3
'''

import heapq

def MST_Prim(graph, V):       
  dist = [float('inf')] * V # Store min-distance from a vertex
  dist[0] = 0 # Set node 0 as starting node of MST

  minHeap = [(0,0)] # Heap to store distances
  visited = set() # Set of already visited nodes
  parent = [-1]*V # List to store parents of nodes     
  
  while minHeap: 
    _, u = heapq.heappop(minHeap) # Extract node "u" that haves smallest weight in min-heap
  
    if u in visited: continue
    visited.add(u)  

    # Run through all adjacent vertices of the extracted vertex u and update their distance values 
    for v in graph[u]: 
      if v in visited: continue
      w = graph[u][v]
      if w < dist[v]: 
        dist[v] = w 
        parent[v] = u 
        # Update distance value in min heap 
        heapq.heappush(minHeap, (w, v))
  
  mst = [ [v, u+1] for u, v in enumerate(parent[1:]) ] # The complete MST
  minCost = sum(dist) # Total weights of the MST
  return mst, minCost